### Evaluate LS performance
Run LS on pseudo and mcts data, evaluate SARI

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import pandas as pd
import numpy as np
from simple_manual import simplify

2025-01-29 14:28:44,093 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-01-29 14:28:46,395 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-01-29 14:28:46,804 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-01-29 14:28:46,804 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-01-29 14:28:46,815 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-01-29 14:28:49,595 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

In [10]:
lines_orig = []
with open('../mcts-main/dataset/mcts.dev.orig', encoding="utf8") as f:
    lines_orig = f.readlines()

In [11]:
lines_ref = []
for dataset in range(0,5):
    filename = str('../mcts-main/dataset/mcts.dev.simp.'+str(dataset))
    with open(filename, encoding="utf8") as f:
        lines_ref.append(f.readlines())

In [17]:
lines_sys = []
for line in lines_orig:
    lines_sys.append(simplify(line,5))

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [21]:
from easse.sari import corpus_sari
from easse.bleu import corpus_bleu

In [22]:
corpus_sari(orig_sents=lines_orig,
    sys_sents=lines_sys,
    refs_sents=lines_ref)
corpus_bleu(sys_sents=lines_sys,
            refs_sents=lines_ref)

0.0